In [1]:
#packages
import pandas as pd
from os import listdir
from os.path import join

In [2]:
#match id of barcelona
path='data/11'
files=listdir(path)

matches_id=[]
for f in files:
    full_path=join(path, f)
    data=pd.read_json(full_path)
    matches_id += list(data['match_id'])

In [3]:
#Extract feature for each feature
def extractFeatureFromDic(x,feature):
    
    if feature in x.keys():
        if feature=='technique':
            return x[feature]['name']
        elif feature=='body_part':
            return x[feature]['name']
        elif feature=='type':
            return x[feature]['name']
        elif feature=='outcome':
            return x[feature]['name']
        elif feature=='freeze_frame':
            return x[feature]
        elif feature=='statsbomb_xg':
            return x[feature]
        elif feature=='first_time':
            return x[feature]
        elif feature=='open_goal':
            return x[feature]
    else:
        return None

In [4]:
#extract features of each event dataframe. Using of extractFeatureFromDic
def preClean(df):
    
    df['shot_bool']=df['type'].apply(lambda x: True if x['id']==16 else False)
    
    mask=df['shot_bool']==True
    shots_df=df[mask]
    
    col=['location', 'under_pressure', 'shot']
    shots_df=shots_df[col]
    
    shots_df['xg']=shots_df['shot'].apply(lambda x: extractFeatureFromDic(x, 'statsbomb_xg'))
    shots_df['first_time']=shots_df['shot'].apply(lambda x: extractFeatureFromDic(x, 'first_time'))
    shots_df['open_goal']=shots_df['shot'].apply(lambda x: extractFeatureFromDic(x, 'open_goal'))
    shots_df['technique']=shots_df['shot'].apply(lambda x: extractFeatureFromDic(x, 'technique'))
    shots_df['body_part']=shots_df['shot'].apply(lambda x: extractFeatureFromDic(x, 'body_part'))
    shots_df['type']=shots_df['shot'].apply(lambda x: extractFeatureFromDic(x, 'type'))
    shots_df['freeze_frame']=shots_df['shot'].apply(lambda x: extractFeatureFromDic(x, 'freeze_frame'))
    shots_df['outcome']=shots_df['shot'].apply(lambda x: extractFeatureFromDic(x, 'outcome'))
    
    shots_df=shots_df.drop('shot', axis=1)
    
    return shots_df

In [5]:
#More cleaning
def secondClean(df):
    
    df['under_pressure']=df['under_pressure'].apply(lambda x: True if x==1.0 else False)
    df['first_time']=df['first_time'].apply(lambda x: True if x==True else False)
    df['open_goal']=df['open_goal'].apply(lambda x: True if x==True else False)
    df['outcome']=df['outcome'].apply(lambda x: True if x=='Goal' else False)
    
    return df

In [7]:
#path of events data
#it's long...
path='https://raw.githubusercontent.com/statsbomb/open-data/master/data/events'

events_df=pd.DataFrame()
for id in matches_id:
    full_path=path + '/' + str(id)+'.json' #os.join() method does not work ...
    df=pd.read_json(full_path)
    clean_df=secondClean(preClean(df))
    #addition of cleaned features of each match to the events dataframe
    events_df=pd.concat([events_df, clean_df])

In [8]:
#save dataframe
events_df.to_pickle('data/clean_df.pickle')